Quantum Convolutional Neural Network

This is a circuit that requires a connectivity map where the distance between the qubits increases exponentially with depth

In [1]:
from qiskit_ibm_runtime.fake_provider import FakeTorino
from qiskit import QuantumCircuit

backend = FakeTorino()

import time
import numpy as np

Define QCNN circuit

In [2]:
num_qubits = 8
num_layers = int(np.ceil(np.log2(num_qubits)))

qc=QuantumCircuit(num_qubits)
i_conv=0
for i_layer in range(num_layers):
    for i_sub_layer in [0 , 2**i_layer]:            
        for i_q1 in range(i_sub_layer, num_qubits, 2**(i_layer+1)):
            i_q2=2**i_layer+i_q1
            if i_q2<num_qubits:
                qc.rxx(np.random.rand(), i_q1, i_q2)
                qc.ry(np.random.rand(), i_q1)
                qc.ry(np.random.rand(), i_q2)
                i_conv+=1

Optionally print out the circuit

In [3]:
qc.draw()

┌───────────────┐┌─────────────┐                                »
q_0: ┤0              ├┤ Ry(0.72314) ├────────────────────────────────»
     │  Rxx(0.55836) │├─────────────┤┌───────────────┐┌─────────────┐»
q_1: ┤1              ├┤ Ry(0.27514) ├┤0              ├┤ Ry(0.54519) ├»
     ├───────────────┤└┬────────────┤│  Rxx(0.98327) │├─────────────┤»
q_2: ┤0              ├─┤ Ry(0.5459) ├┤1              ├┤ Ry(0.45994) ├»
     │  Rxx(0.57048) │┌┴────────────┤└┬──────────────┤├─────────────┤»
q_3: ┤1              ├┤ Ry(0.90931) ├─┤0             ├┤ Ry(0.23376) ├»
     ├───────────────┤├─────────────┤ │  Rxx(0.8317) │├─────────────┤»
q_4: ┤0              ├┤ Ry(0.45813) ├─┤1             ├┤ Ry(0.54314) ├»
     │  Rxx(0.42935) │├─────────────┤┌┴──────────────┤├─────────────┤»
q_5: ┤1              ├┤ Ry(0.61706) ├┤0              ├┤ Ry(0.19748) ├»
     ├───────────────┤├─────────────┤│  Rxx(0.85523) │└┬────────────┤»
q_6: ┤0              ├┤ Ry(0.28059) ├┤1              ├─┤ Ry(0.4966) ├»
     │  Rxx(0.13241) │├─────────────┤└───────────────┘ └────────────┘»
q_7: ┤1              ├┤ Ry(0.90512) ├────────────────────────────────»
     └───────────────┘└─────────────┘                                »
«     ┌───────────────┐┌─────────────┐                                »
«q_0: ┤0              ├┤ Ry(0.55991) ├────────────────────────────────»
«     │               │└─────────────┘                                »
«q_1: ┤  Rxx(0.54909) ├───────────────────────────────────────────────»
«     │               │┌─────────────┐┌───────────────┐┌─────────────┐»
«q_2: ┤1              ├┤ Ry(0.56106) ├┤0              ├┤ Ry(0.80361) ├»
«     └───────────────┘└─────────────┘│               │└─────────────┘»
«q_3: ────────────────────────────────┤  Rxx(0.98369) ├───────────────»
«      ┌─────────────┐ ┌─────────────┐│               │┌─────────────┐»
«q_4: ─┤0            ├─┤ Ry(0.36268) ├┤1              ├┤ Ry(0.89602) ├»
«      │             │ └─────────────┘└───────────────┘└─────────────┘»
«q_5: ─┤  Rxx(0.944) ├────────────────────────────────────────────────»
«      │             │ ┌─────────────┐                                »
«q_6: ─┤1            ├─┤ Ry(0.92553) ├────────────────────────────────»
«      └─────────────┘ └─────────────┘                                »
«q_7: ────────────────────────────────────────────────────────────────»
«                                                                     »
«     ┌───────────────┐┌─────────────┐
«q_0: ┤0              ├┤ Ry(0.62616) ├
«     │               │└─────────────┘
«q_1: ┤               ├───────────────
«     │               │               
«q_2: ┤  Rxx(0.11146) ├───────────────
«     │               │               
«q_3: ┤               ├───────────────
«     │               │┌─────────────┐
«q_4: ┤1              ├┤ Ry(0.78566) ├
«     └───────────────┘└─────────────┘
«q_5: ────────────────────────────────
«                                     
«q_6: ────────────────────────────────
«                                     
«q_7: ────────────────────────────────
«

Compile with default passes

In [4]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
default_pm = generate_preset_pass_manager(backend=backend, optimization_level=3)

t1 = time.time()
default_qc = default_pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = default_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))

Time taken:  0.31359195709228516
OrderedDict({'sx': 94, 'rz': 92, 'cz': 34, 'x': 11})
Number of 2-qubit gates:  34
Number of 1-qubit gates:  197


Create custom compiler

In [5]:
from qiskit.transpiler import PassManager
import qiskit.transpiler.passes as passes 

custom_pm = PassManager()

custom_pm.append(passes.Collect2qBlocks())
custom_pm.append(passes.ConsolidateBlocks())

# custom_pm.append(passes.Decompose())
# custom_pm.append(passes.Optimize1qGatesDecomposition())

custom_qc = custom_pm.run(qc)

gate_counts = custom_qc.count_ops()
print(gate_counts)

custom_qc.draw()

OrderedDict({'ry': 22, 'rxx': 11})


┌───────────────┐┌─────────────┐                                »
q_0: ┤0              ├┤ Ry(0.72314) ├────────────────────────────────»
     │  Rxx(0.55836) │├─────────────┤┌───────────────┐┌─────────────┐»
q_1: ┤1              ├┤ Ry(0.27514) ├┤0              ├┤ Ry(0.54519) ├»
     ├───────────────┤└┬────────────┤│  Rxx(0.98327) │├─────────────┤»
q_2: ┤0              ├─┤ Ry(0.5459) ├┤1              ├┤ Ry(0.45994) ├»
     │  Rxx(0.57048) │┌┴────────────┤└┬──────────────┤├─────────────┤»
q_3: ┤1              ├┤ Ry(0.90931) ├─┤0             ├┤ Ry(0.23376) ├»
     ├───────────────┤├─────────────┤ │  Rxx(0.8317) │├─────────────┤»
q_4: ┤0              ├┤ Ry(0.45813) ├─┤1             ├┤ Ry(0.54314) ├»
     │  Rxx(0.42935) │├─────────────┤┌┴──────────────┤├─────────────┤»
q_5: ┤1              ├┤ Ry(0.61706) ├┤0              ├┤ Ry(0.19748) ├»
     ├───────────────┤├─────────────┤│  Rxx(0.85523) │└┬────────────┤»
q_6: ┤0              ├┤ Ry(0.28059) ├┤1              ├─┤ Ry(0.4966) ├»
     │  Rxx(0.13241) │├─────────────┤└───────────────┘ └────────────┘»
q_7: ┤1              ├┤ Ry(0.90512) ├────────────────────────────────»
     └───────────────┘└─────────────┘                                »
«     ┌───────────────┐┌─────────────┐                                »
«q_0: ┤0              ├┤ Ry(0.55991) ├────────────────────────────────»
«     │               │└─────────────┘                                »
«q_1: ┤  Rxx(0.54909) ├───────────────────────────────────────────────»
«     │               │┌─────────────┐┌───────────────┐┌─────────────┐»
«q_2: ┤1              ├┤ Ry(0.56106) ├┤0              ├┤ Ry(0.80361) ├»
«     └───────────────┘└─────────────┘│               │└─────────────┘»
«q_3: ────────────────────────────────┤  Rxx(0.98369) ├───────────────»
«      ┌─────────────┐ ┌─────────────┐│               │┌─────────────┐»
«q_4: ─┤0            ├─┤ Ry(0.36268) ├┤1              ├┤ Ry(0.89602) ├»
«      │             │ └─────────────┘└───────────────┘└─────────────┘»
«q_5: ─┤  Rxx(0.944) ├────────────────────────────────────────────────»
«      │             │ ┌─────────────┐                                »
«q_6: ─┤1            ├─┤ Ry(0.92553) ├────────────────────────────────»
«      └─────────────┘ └─────────────┘                                »
«q_7: ────────────────────────────────────────────────────────────────»
«                                                                     »
«     ┌───────────────┐┌─────────────┐
«q_0: ┤0              ├┤ Ry(0.62616) ├
«     │               │└─────────────┘
«q_1: ┤               ├───────────────
«     │               │               
«q_2: ┤  Rxx(0.11146) ├───────────────
«     │               │               
«q_3: ┤               ├───────────────
«     │               │┌─────────────┐
«q_4: ┤1              ├┤ Ry(0.78566) ├
«     └───────────────┘└─────────────┘
«q_5: ────────────────────────────────
«                                     
«q_6: ────────────────────────────────
«                                     
«q_7: ────────────────────────────────
«

In [6]:
# Get gate counts
gate_counts = custom_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0))
print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))


OrderedDict({'ry': 22, 'rxx': 11})
Number of 2-qubit gates:  0
Number of 1-qubit gates:  0


In [7]:
#Test with UCC transpiler
import sys
sys.path.append('../')

import transpiler.ucc_default as ucc_default

ucc_transpiler = ucc_default.UCCDefault1()
t1 = time.time()
ucc_qc = ucc_transpiler.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = ucc_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cx", 0))
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

Time taken:  0.015089988708496094
OrderedDict({'unitary': 11})
Number of 2-qubit gates:  0
Number of 1-qubit gates:  0


In [8]:
print(ucc_qc)

     ┌──────────┐            ┌──────────┐            ┌──────────┐
q_0: ┤0         ├────────────┤0         ├────────────┤0         ├
     │  Unitary │┌──────────┐│          │            │          │
q_1: ┤1         ├┤0         ├┤  Unitary ├────────────┤          ├
     ├──────────┤│  Unitary ││          │┌──────────┐│          │
q_2: ┤0         ├┤1         ├┤1         ├┤0         ├┤  Unitary ├
     │  Unitary │├──────────┤└──────────┘│          ││          │
q_3: ┤1         ├┤0         ├────────────┤  Unitary ├┤          ├
     ├──────────┤│  Unitary │┌──────────┐│          ││          │
q_4: ┤0         ├┤1         ├┤0         ├┤1         ├┤1         ├
     │  Unitary │├──────────┤│          │└──────────┘└──────────┘
q_5: ┤1         ├┤0         ├┤  Unitary ├────────────────────────
     ├──────────┤│  Unitary ││          │                        
q_6: ┤0         ├┤1         ├┤1         ├────────────────────────
     │  Unitary │└──────────┘└──────────┘                        
q_7: ┤1   